In [11]:
from pprint import pprint

In [2]:
with open("./stored_authors/author_url_pairings.txt", encoding="utf8") as pairings_file:
    author_url_pairings = eval(pairings_file.read())

In [7]:
with open("./stored_authors/unknown_authors.txt") as unknowns_file:
    unknown_authors = eval(unknowns_file.read())

In [8]:
with open("./stored_authors/duplicate_authors.txt") as duplicates_file:
    duplicate_authors = eval(duplicates_file.read())

In [19]:
unknown_pairing = {}

for unknown in unknown_authors:
    unknown_pairing[unknown] = None

In [22]:
for unknown, pair in unknown_authors.items():
    
    if unknown in author_url_pairings:
        unknown_url = author_url_pairings[unknown]
    else:
        unknown_url = "no"
    
#     print(unknown_url)
    
    found = "Unique"
    for pair_id in pair:
        pair_url = author_url_pairings[pair_id]
#         print(pair_url)
        if unknown_url ==  pair_url:
            found = pair_id
    unknown_pairing[unknown] = found

no
https://www.researchgate.net/scientific-contributions/82451333_J_R_Johnson
https://www.researchgate.net/profile/Andrew_Allen4
https://www.researchgate.net/profile/Andrew_Allen4
https://www.researchgate.net/scientific-contributions/69138982_ZH_Zhu
https://www.researchgate.net/scientific-contributions/2008399536_Zhaoqi_Zhu
https://www.researchgate.net/scientific-contributions/39172246_Zhonghua_Zhu
https://www.researchgate.net/scientific-contributions/2003043257_M_Smith
https://www.researchgate.net/scientific-contributions/39348518_Mark_D_Smith
https://www.researchgate.net/profile/Martin_Smith33
https://www.researchgate.net/scientific-contributions/11707723_Milton_R_Smith
https://www.researchgate.net/profile/Rami_Chahine2
NOT_FOUND2
https://www.researchgate.net/profile/V_Rudolph
https://www.researchgate.net/profile/V_Rudolph
https://www.researchgate.net/profile/Darren_Bradshaw
https://www.researchgate.net/profile/Darren_Bradshaw
https://www.researchgate.net/profile/Toshihiko_Fujimori
h

In [21]:
pprint(unknown_pairing)

{'0000-0002-3608-8003': 0,
 '0000-0002-6496-8411': 'b7c2c5fbb917762d934fdaec1a6c4f7272ea3254',
 '00ae0596cc30940fcc470ce7acfd18f8fcede4ef': 0,
 '019bb48ca9ec40a0f221ed451ee9de2383f12538': 0,
 '020a08bc0da60eefe9886f8cf54a14db1f1a43ab': 0,
 '02470cc05fe10488eae25b981d88bba34b3c80be': '966bbe37507c538696f134495ca6d67b360b64a2',
 '036e372dcae3263b44e3566b0afb897c09843add': '624566d921f2ced905a2803e3b30a08a990ddafc',
 '04738a3e4ecb3c4b065063280d614143956e7331': '52b9ec426c110236605026615b0db869a094b255',
 '04b68c3604022473f9658d2bf55a583665efd86e': '0000-0002-2980-7997',
 '063afc79a74f1383328010539749041711aff4e6': 0,
 '07a9b404b4f1141e0a5e3d2e361757316f6a4599': 0,
 '0816ea96c10155d9218910368ca365d1365ec562': 0,
 '088d865ab8b54217844103b6472ac6c3a571d9bb': 0,
 '096a326cd38289ea0bf9b8623ba57dff97d3cd80': '92d16c28f557037ca62ba48feae99f6c86820fbc',
 '09e88eba05218f20aa99885a2ca16571a999dc74': 'b2106bdb4c8425c5650d24f679b3cd98b3c6b4a0',
 '09ec97d529c4939a0cbc9bce8918f5498c02409a': 0,
 '0adde8

In [6]:
print(author_url_pairings[sorted(author_url_pairings)[:1][0]])

https://www.researchgate.net/profile/Y_Gensterblum
